In [1]:
import os  # For interacting with the operating system (e.g., file paths)
import pandas as pd  # For loading and manipulating CSV data (bounding boxes and image info)
import yaml
import shutil
import numpy as np  # For numerical operations and array handling
import cv2  # For image processing and manipulation (OpenCV library)
import matplotlib.pyplot as plt  # For visualizing data and images (plots)
import seaborn as sns  # For advanced data visualization (especially histograms and distributions)
from glob import glob  # For finding all image files in a directory (using wildcard patterns)
from sklearn.model_selection import train_test_split  # For splitting the dataset into training and testing sets
from PIL import Image
from ultralytics import YOLO


import warnings
warnings.filterwarnings("ignore", "use_inf_as_na option is deprecated")

In [2]:
root_dir = "./kaggle/working/data"
labels_dir = "./kaggle/working/data/labels"
images_dir = "./kaggle/working/data/images"

train_data = r"./kaggle/versions/2/data/training_images"
csv_data = r"./kaggle/versions/2/data/train_solution_bounding_boxes (1).csv"
test_data = r"./kaggle/versions/2/data/testing_images"

In [ ]:
# Loading the CSV data
df = pd.read_csv(csv_data)

# Display the first few rows of the dataframe to understand its structure
df.head()

In [ ]:
# Visualizing sample images with bounding boxes
def plot_images_with_boxes(image_path, df, n=5):
    images = glob(f'{image_path}/*.jpg')[:n]
    for img_path in images:
        img = cv2.imread(img_path)
        img_name = os.path.basename(img_path)
        boxes = df[df['image'] == img_name]  # تغيير 'filename' إلى 'image'

        for _, box in boxes.iterrows():
            cv2.rectangle(img,
                          (int(box['xmin']), int(box['ymin'])),
                          (int(box['xmax']), int(box['ymax'])),
                          (0, 255, 0), 2)

        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(img_name)
        plt.axis('off')
        plt.show()

In [ ]:
plot_images_with_boxes(train_data, df)

In [ ]:
def voc_to_yolo(size, box):
    img_width, img_height = size
    xmin, ymin, xmax, ymax = box

    # Calculate center coordinates
    center_x = (xmin + xmax) / 2.0
    center_y = (ymin + ymax) / 2.0

    # Calculate box dimensions
    box_width = xmax - xmin
    box_height = ymax - ymin

    # Normalize coordinates
    normalized_center_x = center_x / img_width
    normalized_center_y = center_y / img_height
    normalized_box_width = box_width / img_width
    normalized_box_height = box_height / img_height

    # Class ID is 0 for 'car'
    class_id = 0

    return (class_id, normalized_center_x, normalized_center_y, normalized_box_width, normalized_box_height)

In [ ]:
# Get a list of all unique image filenames
unique_images = df['image'].unique()

# Loop through each unique image filename
for image_name in unique_images:
    image_boxes = df[df['image'] == image_name]

In [ ]:
# Get a list of all unique image filenames
unique_images = df['image'].unique()

# Loop through each unique image filename
for image_name in unique_images:
    image_boxes = df[df['image'] == image_name]

    image_path = os.path.normpath(os.path.join(train_data, image_name))

    img = cv2.imread(image_path)

    img_height, img_width, _ = img.shape

    image_size = (img_width, img_height)

    print(f"Processing image: {image_name} with dimensions: {image_size}")

In [ ]:
# Get a list of all unique image filenames
unique_images = df['image'].unique()

# Loop through each unique image filename
for image_name in unique_images:
    image_boxes = df[df['image'] == image_name]

    image_path = os.path.join(train_data, image_name)

    img = cv2.imread(image_path)

    img_height, img_width, _ = img.shape

    annotation_filename = image_name.replace('.jpg', '.txt')
    annotation_path = os.path.join(labels_dir, 'train', annotation_filename)

    with open(annotation_path, 'w') as f:
        for _, box in image_boxes.iterrows():
            xmin, ymin, xmax, ymax = box[['xmin', 'ymin', 'xmax', 'ymax']]

            yolo_coords = voc_to_yolo((img_width, img_height), (xmin, ymin, xmax, ymax))

            yolo_string = f"{yolo_coords[0]} {yolo_coords[1]:.6f} {yolo_coords[2]:.6f} {yolo_coords[3]:.6f} {yolo_coords[4]:.6f}\n"

            f.write(yolo_string)

    print(f"Generated YOLO annotation for {image_name} at {annotation_path}")

In [ ]:
# Get a list of all unique image filenames
unique_images = df['image'].unique()

# Split the unique image filenames into training and validation sets
train_images, val_images = train_test_split(unique_images, test_size=0.2, random_state=42)

print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")

In [ ]:
# Organize training images and labels
for image_name in train_images:
    # Copy image
    src_image_path = os.path.join(train_data, image_name)
    dest_image_path = os.path.join(images_dir, 'train', image_name)
    shutil.copyfile(src_image_path, dest_image_path)

# Organize validation images and labels
for image_name in val_images:
    src_image_path = os.path.join(train_data, image_name)
    dest_image_path = os.path.join(images_dir, 'val', image_name)
    shutil.copyfile(src_image_path, dest_image_path)

    # Copy label from train labels directory to val labels directory
    annotation_filename = image_name.replace('.jpg', '.txt')
    src_annotation_path = os.path.join(labels_dir, 'train', annotation_filename)
    dest_annotation_path = os.path.join(labels_dir, 'val', annotation_filename)
    shutil.copyfile(src_annotation_path, dest_annotation_path)

print("Image and label files organized into train and validation directories.")

In [ ]:
# Define the content of the data.yaml file
data_yaml_content = {
    'train': '../images/train',
    'val': '../images/val',
    'nc': 1,
    'names': ['car']
}

# Construct the full path for the data.yaml file
data_yaml_path = os.path.join(root_dir, 'data.yaml')

# Write the dictionary to the data.yaml file
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml_content, f)

print(f"data.yaml file created at: {data_yaml_path}")

In [3]:
model = YOLO('yolo11n.pt')

In [4]:
results = model.train(
    data = './kaggle/working/data/data.yaml',
    epochs = 10,
    imgsz = 640,
    batch = 16,
    name = 'car_detection_modelv11'
)

New https://pypi.org/project/ultralytics/8.3.184 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.170  Python-3.12.6 torch-2.7.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./kaggle/working/data/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=car_detection_modelv11, nbs=64, nms=False, 

train: Scanning C:\Users\ervin\Documents\img-assignment\kaggle\working\data\labels\train.cache... 284 images, 0 backgrounds, 0 corrupt: 100%|█████████

val: Fast image access  (ping: 0.20.1 ms, read: 134.222.3 MB/s, size: 103.4 KB)



c:\Users\ervin\Documents\img-assignment\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\ervin\Documents\img-assignment\kaggle\working\data\labels\val.cache... 71 images, 0 backgrounds, 0 corrupt: 100%|██████████| 71

Plotting labels to runs\detect\car_detection_modelv11\labels.jpg... 


...

      10/10         0G      1.126       1.15      1.064         16        640: 100%|██████████| 18/18 [02:12<00:00,  7.34s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<00:00,  2.32s/it]

                   all         71        119      0.983      0.933      0.976      0.648



10 epochs completed in 0.394 hours.
Optimizer stripped from runs\detect\car_detection_modelv11\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\car_detection_modelv11\weights\best.pt, 5.4MB

Validating runs\detect\car_detection_modelv11\weights\best.pt...
Ultralytics 8.3.170  Python-3.12.6 torch-2.7.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<00:00,  1.91s/it]


                   all         71        119      0.983      0.933      0.976      0.649
Speed: 1.1ms preprocess, 63.9ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to runs\detect\car_detection_modelv11


In [5]:
# Evaluate the model on the validation set
metrics = model.val()
print(metrics)

Ultralytics 8.3.170  Python-3.12.6 torch-2.7.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 338.998.6 MB/s, size: 90.7 KB)


val: Scanning C:\Users\ervin\Documents\img-assignment\kaggle\working\data\labels\val.cache... 71 images, 0 backgrounds, 0 corrupt: 100%|██████████| 71
c:\Users\ervin\Documents\img-assignment\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.15s/it]


                   all         71        119      0.983      0.933      0.976      0.649
Speed: 1.0ms preprocess, 61.5ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to runs\detect\car_detection_modelv112
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002184288CB00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,

In [6]:
test_image_path = os.path.join(test_data, 'vid_5_26620.jpg')
results = model(test_image_path)

results[0].show()


image 1/1 C:\Users\ervin\Documents\img-assignment\kaggle\versions\2\data\testing_images\vid_5_26620.jpg: 384x640 1 car, 227.2ms
Speed: 2.5ms preprocess, 227.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)


In [7]:
# Get a list of all image filenames in the test directory
test_images = glob(f'{test_data}/*.jpg')

# Select 30 random image paths from the test data
random_test_images = np.random.choice(test_images, size=30, replace=False)

# Function to display images with predictions
def plot_predictions(image_paths, model):
    for img_path in image_paths:
        # Perform inference
        results = model(img_path)

        # Display the results
        results[0].show()

# Plot predictions on the random test images
plot_predictions(random_test_images, model)


image 1/1 C:\Users\ervin\Documents\img-assignment\kaggle\versions\2\data\testing_images\vid_5_26660.jpg: 384x640 1 car, 123.4ms
Speed: 3.8ms preprocess, 123.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\ervin\Documents\img-assignment\kaggle\versions\2\data\testing_images\vid_5_28580.jpg: 384x640 (no detections), 159.0ms
Speed: 6.4ms preprocess, 159.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\ervin\Documents\img-assignment\kaggle\versions\2\data\testing_images\vid_5_31260.jpg: 384x640 (no detections), 273.0ms
Speed: 5.9ms preprocess, 273.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\ervin\Documents\img-assignment\kaggle\versions\2\data\testing_images\vid_5_25180.jpg: 384x640 (no detections), 495.4ms
Speed: 4.7ms preprocess, 495.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\ervin\Documents\img-assignment\kaggle\vers

In [8]:
model.export()

Ultralytics 8.3.170  Python-3.12.6 torch-2.7.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)

PyTorch: starting from 'runs\detect\car_detection_modelv11\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)

TorchScript: starting export with torch 2.7.1+cpu...
TorchScript: export success  5.4s, saved as 'runs\detect\car_detection_modelv11\weights\best.torchscript' (10.4 MB)

Export complete (6.4s)
Results saved to C:\Users\ervin\Documents\img-assignment\runs\detect\car_detection_modelv11\weights
Predict:         yolo predict task=detect model=runs\detect\car_detection_modelv11\weights\best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs\detect\car_detection_modelv11\weights\best.torchscript imgsz=640 data=./kaggle/working/data/data.yaml  
Visualize:       https://netron.app


'runs\\detect\\car_detection_modelv11\\weights\\best.torchscript'